In [ ]:
# Synthetic dataset for multi-class classification of support tickets
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

np.random.seed(42)

labels = ["billing", "login", "bug", "feature_request", "cancellation", "other"]

examples = [
    ("I was charged twice for my subscription this month.", "billing"),
    ("Payment failed but money was deducted.", "billing"),
    ("Can't reset my password, the email never arrives.", "login"),
    ("2FA code is not working and I'm locked out.", "login"),
    ("App crashes every time I open the settings page.", "bug"),
    ("Search results are empty even for common terms.", "bug"),
    ("Please add dark mode and font size controls.", "feature_request"),
    ("Would love an export to CSV option in reports.", "feature_request"),
    ("I want to cancel my plan effective immediately.", "cancellation"),
    ("How do I delete my account and cancel renewal?", "cancellation"),
    ("Where can I find your privacy policy?", "other"),
    ("Do you support Hindi language UI?", "other"),
]

# Expand to ~60 rows by adding slight variations
rows = []
for i in range(60):
    text, y = examples[np.random.randint(0, len(examples))]
    # tiny noise
    text2 = text + (" Please help." if np.random.rand() < 0.4 else "")
    rows.append((text2, y))

df = pd.DataFrame(rows, columns=["ticket_text", "true_label"])
df.head()

In [ ]:
# Create prompt template
PROMPT = """
You are a customer support triage assistant.
Classify the ticket into exactly one category from:
[billing, login, bug, feature_request, cancellation, other]

Return ONLY the label.
Ticket:
"""
print(PROMPT)

In [ ]:
# Mock LLM prediction function
def mock_llm_predict(text):
    t = text.lower()
    if any(k in t for k in ["charged", "payment", "refund", "billing", "deducted"]):
        return "billing"
    if any(k in t for k in ["password", "login", "locked", "2fa", "otp"]):
        return "login"
    if any(k in t for k in ["crash", "bug", "error", "broken", "not working"]):
        return "bug"
    if any(k in t for k in ["add", "feature", "would love", "please include", "dark mode", "export"]):
        return "feature_request"
    if any(k in t for k in ["cancel", "cancellation", "delete my account", "stop renewal"]):
        return "cancellation"
    return "other"

df["pred_label_v1"] = df["ticket_text"].apply(mock_llm_predict)
df.head()

In [ ]:
# Confusion matrix
print(classification_report(df["true_label"], df["pred_label_v1"], digits=3))

In [ ]:
# Confusion matrix
cm = confusion_matrix(df["true_label"], df["pred_label_v1"], labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)
plt.title("Confusion matrix: mock LLM v1")
plt.tight_layout()
plt.show()

In [ ]:
# Show some error examples
errors = df[df["true_label"] != df["pred_label_v1"]].copy()
errors.head(20)

In [ ]:
# Most common error types
errors.groupby(["true_label", "pred_label_v1"]).size().sort_values(ascending=False).head(10)

In [ ]:
# Improved mock LLM prediction function v2
def mock_llm_predict_v2(text):
    t = text.lower()
    # prioritize cancellation keywords
    if any(k in t for k in ["cancel", "cancellation", "delete my account", "close my account", "stop renewal"]):
        return "cancellation"
    if any(k in t for k in ["charged", "payment", "refund", "billing", "deducted"]):
        return "billing"
    if any(k in t for k in ["password", "login", "locked", "2fa", "otp"]):
        return "login"
    if any(k in t for k in ["crash", "bug", "error", "broken", "not working"]):
        return "bug"
    if any(k in t for k in ["dark mode", "export", "feature", "would love", "please add"]):
        return "feature_request"
    return "other"

df["pred_label_v2"] = df["ticket_text"].apply(mock_llm_predict_v2)

print("V1 report:\n", classification_report(df["true_label"], df["pred_label_v1"], digits=3))
print("V2 report:\n", classification_report(df["true_label"], df["pred_label_v2"], digits=3))